In [ ]:
!pip install tiktoken datasets --quiet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import tiktoken
from tqdm import tqdm

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
with open("wikitext2.txt", "w", encoding="utf-8") as f:
    for item in dataset["train"]:
        f.write(item["text"].replace("\n", " ") + "\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
N = 1000  # number of lines
with open("wikitext2.txt", "r", encoding="utf-8") as fin, open("wikitext2_small.txt", "w", encoding="utf-8") as fout:
    for i, line in enumerate(fin):
        if i >= N:
            break
        fout.write(line)

In [ ]:
class TiktokenDataset(Dataset):
    def __init__(self, filepath, tokenizer, block_size):
        with open(filepath, 'r', encoding='utf-8') as f:
            data = f.read()
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.encoded = tokenizer.encode(data)

    def __len__(self):
        return max(0, len(self.encoded) - self.block_size)

    def __getitem__(self, idx):
        chunk = self.encoded[idx:idx+self.block_size+1]
        x = torch.tensor(chunk[:-1], dtype=torch.long)
        y = torch.tensor(chunk[1:], dtype=torch.long)
        return x, y

In [ ]:
class GPTConfig:
    def __init__(
        self,
        vocab_size,
        block_size,
        n_layers=4,
        n_heads=4,
        n_embd=128,
        dropout=0.1,
        bias=True,
        tie_weights=True,
        use_learnable_pos_emb=True,
    ):
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.n_layers = n_layers
        self.n_heads = n_heads
        self.n_embd = n_embd
        self.dropout = dropout
        self.bias = bias
        self.tie_weights = tie_weights
        self.use_learnable_pos_emb = use_learnable_pos_emb

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_heads == 0
        self.n_heads = config.n_heads
        self.head_dim = config.n_embd // config.n_heads
        self.scale = self.head_dim ** -0.5

        self.qkv = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        self.proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        self.attn_drop = nn.Dropout(config.dropout)
        self.resid_drop = nn.Dropout(config.dropout)

        self.register_buffer("mask", torch.tril(torch.ones(config.block_size, config.block_size)).unsqueeze(0).unsqueeze(0))

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.qkv(x)
        q, k, v = qkv.split(C, dim=2)
        q = q.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) * self.scale
        att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.resid_drop(self.proj(y))
        return y

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.dropout),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.tok_emb = nn.Embedding(config.vocab_size, config.n_embd)
        if config.use_learnable_pos_emb:
            self.pos_emb = nn.Parameter(torch.zeros(1, config.block_size, config.n_embd))
        else:
            self.register_buffer('pos_emb', self._get_sinusoidal_pos_emb(config.block_size, config.n_embd))
        self.drop = nn.Dropout(config.dropout)
        self.blocks = nn.ModuleList([Block(config) for _ in range(config.n_layers)])
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=config.bias)
        if config.tie_weights:
            self.head.weight = self.tok_emb.weight
        self.block_size = config.block_size
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def _get_sinusoidal_pos_emb(self, seq_len, dim):
        position = torch.arange(seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim, 2).float() * (-torch.log(torch.tensor(10000.0)) / dim))
        pe = torch.zeros(seq_len, dim)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        assert T <= self.block_size, "Sequence too long"
        tok_emb = self.tok_emb(idx)
        if self.config.use_learnable_pos_emb:
            pos_emb = self.pos_emb[:, :T, :]
        else:
            pos_emb = self.pos_emb[:, :T, :].to(tok_emb.device)
        x = self.drop(tok_emb + pos_emb)
        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x)
        logits = self.head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
block_size = 32
dataset_file = "wikitext2_small.txt"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset = TiktokenDataset(dataset_file, tokenizer, block_size)
if len(dataset) == 0:
    raise ValueError("Dataset is empty! Check your dataset file and block_size.")

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

block_size = 32
batch_size = 4

config = GPTConfig(
    vocab_size=tokenizer.n_vocab,
    block_size=block_size,
    n_layers=1,
    n_heads=1,
    n_embd=32,
    dropout=0.1,
    tie_weights=True,
    use_learnable_pos_emb=True,
)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
model = GPT(config).to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

for epoch in range(1):  # Increase epochs for real training
    model.train()
    total_loss = 0
    for x, y in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
        x, y = x.to(device), y.to(device)
        logits, loss = model(x, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

# Save the trained model
torch.save(model.state_dict(), 'gpt_tiktoken.pth')

Epoch 1: 100%|██████████| 15616/15616 [27:28<00:00,  9.47it/s]

Epoch 1, Loss: 3.5361


In [ ]:
def sample(model, tokenizer, start_text, length=100, temperature=1.0):
    model.eval()
    idx = torch.tensor([tokenizer.encode(start_text)], dtype=torch.long).to(next(model.parameters()).device)
    for _ in range(length):
        idx_cond = idx[:, -model.block_size:]
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / temperature
        probs = torch.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat([idx, next_id], dim=1)
    return tokenizer.decode(idx[0].tolist())

print(sample(model, tokenizer, start_text="Once upon a time", length=50))

Once upon a time leadership CD , but traded for his action of the fourth two could be blamed for many Native Americans who represented a god was discontinued and installing a young chaos . Egypt , which also never statements at chaos in the gods died of creation of the war of the


In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
with open("wikitext2.txt", "r", encoding="utf-8") as f:
    data = f.read()
encoded = tokenizer.encode(data)
print("Number of tokens:", len(encoded))

Number of tokens: 2416842


In [ ]:
!ls -lh wikitext2.txt

-rw-r--r-- 1 root root 11M Aug  1 05:53 wikitext2.txt


In [ ]:
block_size = 128
print("Number of samples:", len(encoded) - block_size)

Number of samples: 2416714


In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
block_size=32
config = GPTConfig(
    vocab_size=tokenizer.n_vocab,
    block_size=block_size,
    n_layers=1,
    n_heads=1,
    n_embd=32,
    dropout=0.1,
    tie_weights=True,
    use_learnable_pos_emb=True,
)
device="cuda"
model = GPT(config)
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

Number of parameters: 1672273
